In [12]:
import pandas as pd
import nltk
from nltk.corpus import stopwords

from collections import Counter
import re

In [13]:
# https://github.com/sinking8/usc-x-24-us-election.git

#Part 1:

chunk_1 = pd.read_csv('./chunks/aug_chunk_1.csv')


C:\Users\sofia\AppData\Local\Temp\ipykernel_18712\3906387663.py:5: DtypeWarning: Columns (7,23,30) have mixed types. Specify dtype option on import or set low_memory=False.
  chunk_1 = pd.read_csv('./chunks/aug_chunk_1.csv')


In [14]:
chunk_1.head()


,type,id,username,text,url,epoch,media,retweetedTweet,retweetedTweetID,retweetedUserID,...,links,viewCount,quotedTweet,in_reply_to_screen_name,in_reply_to_status_id_str,in_reply_to_user_id_str,location,cash_app_handle,user,0
0,tweet-,1.842096e+18,Debbie133467421,Kamala Harris is very connected to Diddy. She ...,https://twitter.com/Debbie133467421/status/184...,1.728025e+09,[],False,NaN,NaN,...,[{'display_url': 'youtube.com/shorts/WTYBwdp…'...,{'state': 'Enabled'},False,NaN,NaN,NaN,NaN,NaN,"{'id': 1820716370360975360, 'id_str': '1820716...",NaN
1,tweet-,1.842096e+18,EMGENT_007,@iamnot_elon 100% Sir!,https://twitter.com/EMGENT_007/status/18420955...,1.728025e+09,[],False,NaN,NaN,...,[],{'state': 'Enabled'},False,iamnot_elon,1.842041e+18,1.652734e+18,NaN,NaN,"{'id': 1772432144033521664, 'id_str': '1772432...",NaN
2,tweet-,1.842096e+18,wellyworldfl,@catturd2 And all the insurance companies will...,https://twitter.com/wellyworldfl/status/184209...,1.728025e+09,[],False,NaN,NaN,...,[],{'state': 'Enabled'},False,catturd2,1.841787e+18,1.043186e+18,NaN,NaN,"{'id': 156726345, 'id_str': '156726345', 'url'...",NaN
3,tweet-,1.842096e+18,emi79907,@CatholicQuote12 BEAUTIFUL 🙏🙏🙏🙏,https://twitter.com/emi79907/status/1842095589...,1.728025e+09,[],False,NaN,NaN,...,[],{'state': 'Enabled'},False,CatholicQuote12,1.841753e+18,1.794233e+18,NaN,NaN,"{'id': 1765294849471676416, 'id_str': '1765294...",NaN
4,tweet-,1.842096e+18,hope_neverLost1,9 Jahre Haft. #TinaPeters hatte ein privilegie...,https://twitter.com/hope_neverLost1/status/184...,1.728025e+09,[],False,NaN,NaN,...,[],{'state': 'Enabled'},True,NaN,NaN,NaN,NaN,NaN,"{'id': 52490268, 'id_str': '52490268', 'url': ...",NaN


In [15]:
#Important columns: 'text', 'user', 
columns = ['text', 'user']

chunk = chunk_1[columns]

In [16]:
chunk.head()

,text,user
0,Kamala Harris is very connected to Diddy. She ...,"{'id': 1820716370360975360, 'id_str': '1820716..."
1,@iamnot_elon 100% Sir!,"{'id': 1772432144033521664, 'id_str': '1772432..."
2,@catturd2 And all the insurance companies will...,"{'id': 156726345, 'id_str': '156726345', 'url'..."
3,@CatholicQuote12 BEAUTIFUL 🙏🙏🙏🙏,"{'id': 1765294849471676416, 'id_str': '1765294..."
4,9 Jahre Haft. #TinaPeters hatte ein privilegie...,"{'id': 52490268, 'id_str': '52490268', 'url': ..."


In [17]:
#change text column to string
#A value is trying to be set on a copy of a slice from a DataFrame.
#Try using .loc[row_indexer,col_indexer] = value instead
chunk.loc[:,'text'] = chunk['text'].astype(str)

In [18]:
chunk_copy = chunk.copy()

In [19]:
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('punkt_tab')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sofia\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\sofia\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\sofia\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\sofia\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [41]:
stop_words = set(stopwords.words('english'))

# Additional common words to ignore can be added to the stop words
additional_stopwords = {'http', 'https', 'co', 'com', 'rt', 'biden', 'trump', 'maga', 'joe', 'gop', 'bidens', 'president', 'kamala', 'harris', 'kamala harris', 'democrat', 'democratic', 'democrats', 'donald', 'republican', 'former' }
stop_words.update(additional_stopwords)

In [32]:
# Function to clean and tokenize text
def clean_tokenize(text):
   # Remove URLs, hashtags, mentions, special characters, and numbers
   text = re.sub(r"(http\S+)|(#\S+)|(@\S+)|[^a-zA-Z\s]|(\w*\d\w*)", '', text)
   # Convert to lower case
   text = text.lower()
   # Tokenize the text
   words = text.split()
   # Remove stop words
   words = [word for word in words if word not in stop_words]
   return words

In [33]:
def common_words(df):
   df['cleaned_text'] = df['text'].apply(clean_tokenize)
   # Flatten the list of words in all tweets into a single list
   all_words = [word for words_list in df['cleaned_text'] for word in words_list]
   # Count the occurrences of each word
   word_counts = Counter(all_words)
   # Get the most common words
   most_common_words = word_counts.most_common(100)  # Adjust the number as needed
   # Display the most common words
   for i in range(len(most_common_words)):
       print(f'{most_common_words[i][0]}: {most_common_words[i][1]}')

In [34]:

# Function to clean, tokenize, and filter adjectives
def clean_tokenize2(text):
   # Remove URLs, hashtags, mentions, special characters, and numbers
   text = re.sub(r"(http\S+)|(#\S+)|(@\S+)|[^a-zA-Z\s]|(\w*\d\w*)", '', text)
   # Convert to lower case
   text = text.lower()
   # Tokenize the text
   words = nltk.word_tokenize(text)
   # Part-of-speech tagging
   pos_tags = nltk.pos_tag(words)
   # Filter adjectives, identified by 'JJ' tag
   adjectives = [word for word, tag in pos_tags if tag.startswith('JJ') and word not in stop_words]
   return adjectives


In [35]:
def common_adjectives(df):
   # Apply the cleaning and tokenization function to each tweet
    df['adjectives'] = df['text'].apply(clean_tokenize2)
    # Flatten the list of adjectives in all tweets into a single list
    all_adjectives = [adj for adj_list in df['adjectives'] for adj in adj_list]
    # Count the occurrences of each adjective
    adjective_counts = Counter(all_adjectives)
    # Get the most common adjectives
    most_common_adjectives = adjective_counts.most_common(30)  # Adjust the number as needed
    # Display the most common adjectives
    for i in range(len(most_common_adjectives)):
        print(f'{most_common_adjectives[i][0]}: {most_common_adjectives[i][1]}')

In [42]:
keywords = ['Donald Trump', 'Trump', 'MAGA']
# Filter the DataFrame to include only tweets containing any of the keywords
# The 'na=False' parameter ensures that tweets with 'NaN' text are ignored
filtered_df = chunk_copy[chunk_copy['text'].str.contains('|'.join(keywords), case=False, na=False)]

keywords2 = ['Kamala Harris', 'harris', 'kamala', 'democrat', 'democratic']	
filtered_df2 = chunk_copy[chunk_copy['text'].str.contains('|'.join(keywords2), case=False, na=False)]

In [43]:
print('Most common words in tweets about Trump:')
common_words(filtered_df)



Most common words in tweets about Trump:
like: 1818
people: 1631
amp: 1418
dont: 1380
vote: 1190
trumps: 1135
one: 1075
get: 1065
would: 1035
know: 974
im: 847
us: 829
think: 827
going: 814
hes: 800
never: 715
time: 697
right: 694
election: 691
party: 673
america: 666
thats: 661
see: 658
want: 655
even: 651
youre: 650
make: 629
years: 623
back: 623
said: 617
need: 592
country: 587
cant: 572
go: 571
doesnt: 558
way: 552
cult: 540
lies: 540
good: 520
say: 492
didnt: 490
republicans: 478
voting: 476
de: 467
still: 461
really: 457
campaign: 451
stop: 440
also: 434
win: 432
got: 432
well: 430
many: 428
better: 426
believe: 424
much: 419
every: 416
nothing: 410
take: 407
rally: 393
support: 390
man: 388
old: 382
could: 380
great: 374
plane: 372
love: 367
yes: 362
isnt: 357
another: 354
money: 351
anything: 350
look: 349
thing: 348
hate: 345
conservative: 345
new: 344
real: 341
media: 340
news: 337
day: 337
debate: 330
world: 328
ever: 321
always: 319
theyre: 319
sure: 314
keep: 313
border: 3

C:\Users\sofia\AppData\Local\Temp\ipykernel_18712\3891492744.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cleaned_text'] = df['text'].apply(clean_tokenize)


In [44]:
print('\nMost common adjectives in tweets about Trump:')
common_adjectives(filtered_df)


Most common adjectives in tweets about Trump:
good: 509
many: 428
old: 382
great: 374
conservative: 345
new: 344
real: 341
last: 309
american: 305
white: 295
much: 290
better: 280
bad: 279
right: 259
sure: 249
amp: 242
true: 227
black: 216
big: 208
first: 203
wrong: 200
im: 192
dont: 187
free: 185
best: 179
trumps: 178
political: 175
mechanical: 170
next: 163
stupid: 162


C:\Users\sofia\AppData\Local\Temp\ipykernel_18712\2122204458.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['adjectives'] = df['text'].apply(clean_tokenize2)


In [45]:
print('\nMost common words in tweets about Kamala Harris:')
common_words(filtered_df2)




Most common words in tweets about Kamala Harris:
people: 1029
vote: 962
like: 933
amp: 913
party: 891
bidenharris: 690
dont: 676
us: 614
one: 605
would: 591
get: 559
america: 507
years: 490
know: 486
going: 457
support: 450
country: 442
think: 434
border: 426
even: 413
election: 408
want: 404
administration: 404
im: 392
campaign: 382
thats: 360
time: 352
cant: 351
never: 346
right: 338
said: 338
american: 337
walz: 330
voting: 328
see: 326
back: 306
hes: 302
americans: 298
shes: 296
media: 296
go: 291
make: 291
doesnt: 287
policies: 287
republicans: 286
win: 284
need: 283
vp: 282
youre: 281
nothing: 274
still: 273
many: 271
really: 270
way: 266
voters: 266
illegal: 256
say: 253
debate: 253
done: 251
money: 250
didnt: 249
new: 244
states: 240
well: 240
good: 238
trumps: 236
also: 232
got: 230
much: 230
via: 226
candidate: 225
better: 223
care: 222
stop: 222
left: 218
take: 214
every: 214
since: 213
last: 210
obama: 210
wont: 209
could: 208
conservative: 203
lies: 203
believe: 200
illeg

C:\Users\sofia\AppData\Local\Temp\ipykernel_18712\3891492744.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cleaned_text'] = df['text'].apply(clean_tokenize)


In [46]:
print('\nMost common adjectives in tweets about Kamala Harris:')
common_adjectives(filtered_df2)


Most common adjectives in tweets about Kamala Harris:
american: 337
many: 271
new: 244
illegal: 240
good: 233
last: 210
conservative: 203
bidenharris: 194
amp: 180
black: 173
real: 164
great: 163
much: 161
better: 157
bad: 155
free: 153
political: 150
white: 146
first: 140
old: 126
big: 114
presidential: 107
open: 106
sure: 106
true: 99
liberal: 99
single: 99
united: 99
right: 99
cant: 98


C:\Users\sofia\AppData\Local\Temp\ipykernel_18712\2122204458.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['adjectives'] = df['text'].apply(clean_tokenize2)
